In [1]:
import os 
import streamlit as st 
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
st.title("RAG Application built on Gemini Model")

2025-08-28 14:51:31.146 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:31.745 
  command:

    streamlit run c:\simple_chatbot\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-28 14:51:31.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:31.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [3]:
#1.load llm
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0)

In [4]:
#2.Load Documents
docs = [Document(page_content="LangChain helps build chatbots using LLMs and retrieval.")]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split = text_splitter.split_documents(docs)

In [5]:
#3.embedding + vectorestore 
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.environ["GOOGLE_API_KEY"])
vectordb = Chroma.from_documents(split, embedding=embeddings)

In [6]:
#4.Retrieve 
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [7]:
query = st.chat_input("Say somethings: ")
prompt = query

2025-08-28 14:51:35.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:35.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:35.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:35.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:35.375 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 14:51:35.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
# 5.custom chat prompt template 
Prompt_template = """
You are a helpful AI assistant 
Answer the question based only on given content

Context:
{context}

Question:
{question}

Answer:
"""
QA_PROMPT = PromptTemplate(template=Prompt_template,input_variables=["context","question"])

In [9]:
#6.QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",     # stuff / map_reduce / refine
    chain_type_kwargs={"prompt": QA_PROMPT}
)

In [10]:
#7.Ask any question 
query = "What is Langchain?"
answer = qa_chain.invoke(query)

print("Answer:", answer)

Answer: {'query': 'What is Langchain?', 'result': 'LangChain helps build chatbots using LLMs and retrieval.'}
